## Prepare train_data and test_data

In [1]:
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from hypergbm import make_experiment
from hypernets.tabular.metrics import metric_to_scoring
from sklearn.metrics import get_scorer

In [2]:
train_data = pd.read_csv('datasets/West_Nile_Virus_II/train.csv.gz')
test_data = pd.read_csv('datasets/West_Nile_Virus_II/test.csv.gz')
X_train = train_data.copy()
y_train = X_train.pop('WnvPresent')
X_test = test_data.copy()
y_test = X_test.pop('WnvPresent')

In [3]:
X_train.head()

,Unnamed: 0,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,Latitude1,Longitude1,AddressAccuracy,NumMosquitos
0,6709,"South Cottage Grove Avenue, Chicago, IL, USA",CULEX PIPIENS/RESTUANS,10,S COTTAGE GROVE,T102,"1000 S COTTAGE GROVE, Chicago, IL",41.750498,-87.605294,32.601011,-86.680736,5,6
1,789,"1500 West Webster Avenue, Chicago, IL 60614, USA",CULEX PIPIENS,15,W WEBSTER AVE,T045,"1500 W WEBSTER AVE, Chicago, IL",41.921600,-87.666455,61.302501,-158.775020,8,2
2,6556,"ORD Terminal 5, O'Hare International Airport, ...",CULEX RESTUANS,10,W OHARE AIRPORT,T900,"1000 W OHARE AIRPORT, Chicago, IL",41.974689,-87.890615,34.168219,-111.930907,9,16
3,8170,"ORD Terminal 5, O'Hare International Airport, ...",CULEX RESTUANS,10,W OHARE AIRPORT,T900,"1000 W OHARE AIRPORT, Chicago, IL",41.974689,-87.890615,34.751928,-92.131378,9,2
4,6517,"3700 South Pulaski Road, Chicago, IL 60623, USA",CULEX PIPIENS/RESTUANS,37,S PULASKI RD,T063,"3700 S PULASKI RD, Chicago, IL",41.825610,-87.726549,37.271875,-119.270415,9,10


In [4]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8404 entries, 0 to 8403
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              8404 non-null   int64  
 1   Address                 8404 non-null   object 
 2   Species                 8404 non-null   object 
 3   Block                   8404 non-null   int64  
 4   Street                  8404 non-null   object 
 5   Trap                    8404 non-null   object 
 6   AddressNumberAndStreet  8404 non-null   object 
 7   Latitude                8404 non-null   float64
 8   Longitude               8404 non-null   float64
 9   Latitude1               8404 non-null   float64
 10  Longitude1              8404 non-null   float64
 11  AddressAccuracy         8404 non-null   int64  
 12  NumMosquitos            8404 non-null   int64  
dtypes: float64(4), int64(4), object(5)
memory usage: 853.7+ KB


# Without feature_generation
_best_reward is 0.75

In [5]:
experiment = make_experiment(train_data.copy(),test_data=test_data.copy(),target='WnvPresent',
                             reward_metric='recall',pos_label=1,random_state=8888,max_trials=10,log_level='info',
                             class_balancing='RandomUnderSampling')
estimator = experiment.run()

07-26 16:31:58 I hypernets.u.common.py 138 - 2 class detected, {0, 1}, so inferred as a [binary classification] task
07-26 16:31:58 I hypernets.e.compete.py 1234 - create experiment with ['data_clean', 'drift_detection', 'space_searching', 'final_ensemble']
07-26 16:31:58 I hypernets.e._maker.py 279 - make_experiment with train data:(8404, 13), test data:(2102, 14), eval data:None, target:WnvPresent


ExperimentProcessWidget(initData='{"steps": [{"index": 0, "name": "data_clean", "type": "DataCleanStep", "stat…

07-26 16:31:59 W hypernets.e.compete.py 1251 - X_train['Unnamed: 0', 'Address', 'Species', 'Block', 'Street', 'Trap', 'AddressNumberAndStreet', 'Latitude', 'Longitude', 'Latitude1', 'Longitude1', 'AddressAccuracy', 'NumMosquitos'] and X_test['Unnamed: 0', 'Address', 'Species', 'Block', 'Street', 'Trap', 'AddressNumberAndStreet', 'Latitude', 'Longitude', 'Latitude1', 'Longitude1', 'AddressAccuracy', 'NumMosquitos', 'WnvPresent'] have different columns before data_clean, try fix it.


07-26 16:31:59 I hypernets.e.compete.py 1263 - fit_transform data_clean with columns: ['Unnamed: 0', 'Address', 'Species', 'Block', 'Street', 'Trap', 'AddressNumberAndStreet', 'Latitude', 'Longitude', 'Latitude1', 'Longitude1', 'AddressAccuracy', 'NumMosquitos']
07-26 16:31:59 I hypernets.e.compete.py 305 - data_clean transform train data
07-26 16:31:59 I hypernets.t.data_cleaner.py 326 - drop columns: ['Unnamed: 0']
07-26 16:31:59 I hypernets.e.compete.py 310 - data_clean transform test data
07-26 16:31:59 I hypernets.t.data_cleaner.py 326 - drop columns: ['Unnamed: 0']
07-26 16:31:59 I hypernets.e.compete.py 351 - data_clean keep 12 columns
07-26 16:31:59 I hypernets.e.compete.py 1263 - fit_transform drift_detection with columns: ['Address', 'Species', 'Block', 'Street', 'Trap', 'AddressNumberAndStreet', 'Latitude', 'Longitude', 'Latitude1', 'Longitude1', 'AddressAccuracy', 'NumMosquitos']
07-26 16:31:59 I hypernets.e.compete.py 186 - drift_detection cache_transform: 12 columns kept 

In [6]:
scorer = get_scorer(metric_to_scoring('recall',pos_label=1))
score = scorer(estimator, X_test, y_test)*scorer._sign
score

07-26 16:34:07 I hypernets.t.data_cleaner.py 326 - drop columns: ['Unnamed: 0']


0.75

# Introduce func of feature_generation
-- visit https://github.com/alteryx/featuretools for more datails

In [7]:
from hypernets.tabular.feature_generators import FeatureGenerationTransformer

In [8]:
train_data['latlong1'] = train_data[['Latitude', 'Longitude']].apply(tuple, axis=1)
test_data['latlong1'] = test_data[['Latitude', 'Longitude']].apply(tuple, axis=1)
train_data['latlong2'] = train_data[['Latitude1', 'Longitude1']].apply(tuple, axis=1)
test_data['latlong2'] = test_data[['Latitude1', 'Longitude1']].apply(tuple, axis=1)
X_train = train_data.copy()
y_train = X_train.pop('WnvPresent')
X_test = test_data.copy()
y_test = X_test.pop('WnvPresent')

In [9]:
featureGenerationTransformer = FeatureGenerationTransformer(latlong_cols=['latlong1','latlong2'])
featureGenerationTransformer.fit(pd.DataFrame(train_data[['latlong1','latlong2']]))
X_test_new = featureGenerationTransformer.transform(pd.DataFrame(test_data[['latlong1','latlong2']]))

In [10]:
X_test_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2102 entries, 0 to 2101
Data columns (total 5 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   latlong1                                               2102 non-null   object 
 1   latlong2                                               2102 non-null   object 
 2   GEOHASH__latlong1__                                    2102 non-null   object 
 3   GEOHASH__latlong2__                                    2102 non-null   object 
 4   HAVERSINE__latlong1____latlong2____unit__kilometers__  2102 non-null   float64
dtypes: float64(1), object(4)
memory usage: 98.5+ KB


In [11]:
X_test_new.head()

,latlong1,latlong2,GEOHASH__latlong1__,GEOHASH__latlong2__,HAVERSINE__latlong1____latlong2____unit__kilometers__
e_hypernets_ft_index,,,,,
0,"(41.948167, -87.730698)","(32.601011199999995, -86.6807365)",dp3weskmy945,djf37wc48kyq,1043.476383
1,"(41.903002, -87.68826700000001)","(61.3025006, -158.7750198)",dp3wks5qem9p,b6zhdzz8cykk,5052.504275
2,"(41.974689, -87.890615)","(34.1682185, -111.930907)",dp3wb12021pk,9w06kunkkcd6,2266.646309
3,"(41.869107, -87.69629300000001)","(34.7519275, -92.13137840000002)",dp3whqj90r0r,9ynmxqruxkc4,880.605092
4,"(41.825902, -87.667827)","(37.2718745, -119.2704153)",dp3tvn4jx94p,9qehjcjumuur,2740.466848


# Use feature_generation
_best_reward is 0.802 

In [12]:
experiment = make_experiment(train_data.copy(),eval_data=test_data.copy(),test_data=test_data.copy(),target='WnvPresent',
                             random_state=8888,max_trials=10,reward_metric='recall',class_balancing='RandomUnderSampling',
                             feature_generation=True,
                             feature_generation_text_cols=['latlong1', 'latlong2'],
                             drift_detection=False,
                             pos_label=1,
                             log_level='info',
                            )
estimator = experiment.run()
scorer = get_scorer(metric_to_scoring('recall',pos_label=1))
score = scorer(estimator, X_test, y_test)
score

07-26 16:34:08 I hypernets.u.common.py 138 - 2 class detected, {0, 1}, so inferred as a [binary classification] task
07-26 16:34:08 I hypernets.e.compete.py 1564 - detected datetime columns: []
07-26 16:34:08 I hypernets.e.compete.py 1567 - detected latlong columns: ['latlong1', 'latlong2']
07-26 16:34:08 I hypernets.e.compete.py 1234 - create experiment with ['data_clean', 'feature_generation', 'space_searching', 'final_ensemble']
07-26 16:34:08 I hypernets.e._maker.py 279 - make_experiment with train data:(8404, 15), test data:(2102, 16), eval data:(2102, 15), target:WnvPresent


ExperimentProcessWidget(initData='{"steps": [{"index": 0, "name": "data_clean", "type": "DataCleanStep", "stat…

07-26 16:34:08 W hypernets.e.compete.py 1251 - X_train['Unnamed: 0', 'Address', 'Species', 'Block', 'Street', 'Trap', 'AddressNumberAndStreet', 'Latitude', 'Longitude', 'Latitude1', 'Longitude1', 'AddressAccuracy', 'NumMosquitos', 'latlong1', 'latlong2'] and X_test['Unnamed: 0', 'Address', 'Species', 'Block', 'Street', 'Trap', 'AddressNumberAndStreet', 'Latitude', 'Longitude', 'Latitude1', 'Longitude1', 'AddressAccuracy', 'NumMosquitos', 'WnvPresent', 'latlong1', 'latlong2'] have different columns before data_clean, try fix it.


07-26 16:34:08 I hypernets.e.compete.py 1263 - fit_transform data_clean with columns: ['Unnamed: 0', 'Address', 'Species', 'Block', 'Street', 'Trap', 'AddressNumberAndStreet', 'Latitude', 'Longitude', 'Latitude1', 'Longitude1', 'AddressAccuracy', 'NumMosquitos', 'latlong1', 'latlong2']
07-26 16:34:09 I hypernets.e.compete.py 297 - data_clean cv enabled, so concat train data and eval data
07-26 16:34:09 I hypernets.e.compete.py 305 - data_clean transform train data
07-26 16:34:09 I hypernets.t.data_cleaner.py 326 - drop columns: ['Unnamed: 0']
07-26 16:34:09 I hypernets.e.compete.py 310 - data_clean transform test data
07-26 16:34:09 I hypernets.t.data_cleaner.py 326 - drop columns: ['Unnamed: 0']
07-26 16:34:09 I hypernets.e.compete.py 351 - data_clean keep 14 columns
07-26 16:34:09 I hypernets.e.compete.py 1263 - fit_transform feature_generation with columns: ['Address', 'Species', 'Block', 'Street', 'Trap', 'AddressNumberAndStreet', 'Latitude', 'Longitude', 'Latitude1', 'Longitude1',

0.8020833333333334